### I'm using this script to test out webscraping using beautifulsoup package.

Basic setup and site access.

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

MainURL='https://www.americanrhetoric.com/top100speechesall.html'
MainPage= requests.get(MainURL, headers={'user-agent': 'Mozilla/5.0'})
soup= BeautifulSoup(MainPage.content, 'html.parser')

Obtaining name of the speakers and titles

In [2]:
names = soup.find_all(attrs={"width": "203"})
nameList = [n.get_text(strip=True) for n in names]

def findTitles (x) :
    titles = x.find_all(attrs={"color":"#BA1D01"})
    titleList1 = [n.get_text(strip=True) for n in titles]
    pattern=re.compile("([\r])|([\t])|([\n])|([\ ]{2,})")
    titleListCleaned = [re.sub(pattern, '', str(title)) for title in titleList1]
    titleListCleaned.pop(0)
    return titleListCleaned

Obtaining URL list from main webpage and storing them as list variable.

In [3]:
mainlinks=[a['href'] for a in soup.find_all('a',href=True)   
    if 'off site' in a.text 
    or a['href'].startswith('speeches') and 'PDFFiles' not in a['href']
    or 'Belief and Public Morality' in a.text]

Obtaining where and when the speech is given 

In [4]:
deliveredLists = []
script = []

for i, link in enumerate(mainlinks) : 
    URL='https://www.americanrhetoric.com/'+mainlinks[i]
    Page=requests.get(URL, headers={'user-agent': 'Mozilla/5.0'})
    speechsoup=BeautifulSoup(Page.content, 'html.parser')
    findWhere = speechsoup.find_all(attrs={"color":"#CE0A04"})
    deliveredLists.append([n.get_text(strip=True) for n in findWhere])
    
    findscript=speechsoup.find_all("font",{'face':'Verdana'})
    script.append([n.get_text(strip=True) for n in findscript])
        



In [15]:
print(script[1])

['Vice President Johnson, Mr. Speaker, Mr. Chief \r\nJustice, \r\nPresident\r\nEisenhower, Vice President Nixon, President Truman, reverend clergy, fellow citizens:', 'Vice President Johnson, Mr. Speaker, Mr. Chief Justice, President Eisenhower, Vice President Nixon, President Truman, reverend clergy, fellow citizens: Vice President Johnson, Mr. Speaker, Mr. Chief Justice, President Eisenhower, Vice President Nixon, President Truman, reverend clergy, fellow citizens: We observe today not a victory of party, but a celebration of freedom -- symbolizing an end, as well as a beginning -- signifying renewal, as well as change. For I have sworn before you and Almighty God the same solemn oath our forebears prescribed nearly a century and three-quarters ago. The world is very different now. For man holds in his mortal hands the power to abolish all forms of human poverty and all forms of human life. And yet the same revolutionary beliefs for which our forebears fought are still at issue aroun

In [21]:
cleanedScripts = []

for i, s in enumerate(script) :
    s =' '.join([str(line) for line in s])  
    s =' '.join(s.split())
    cleanedScripts.append(s)



['Thank you ladies and gentlemen for a very warm reception. It was one hundred and forty-four years ago that members of the Democratic Party first met in convention to select a Presidential candidate. Since that time, Democrats have continued to convene once every four years and draft a party platform and nominate a Presidential candidate. And our meeting this week is a continuation of that tradition. But there is something different about tonight. There is something special about tonight. What is different? What is special? I, Barbara Jordan, am a keynote speaker. When -- A lot of years passed since 1832, and during that time it would have been most unusual for any national political party to ask a Barbara Jordan to deliver a keynote address. But tonight, here I am. And I feel -- I feel that notwithstanding the past that my presence here is one additional bit of evidence that the American Dream need not forever be deferred. Thank you ladies and gentlemen for a very warm reception. It 

In [22]:
deliveredAtCleaned = []

for i, place in enumerate(deliveredLists) : 
    if len(place) == 0 : 
      deliveredAtCleaned.append(None) 
      pattern=re.compile("([\r])|([\t])|([\n])|([\ ]{2,})")
      deliveredAtCleaned = [re.sub(pattern, '', str(delPlace)) for delPlace in deliveredAtCleaned]
    else : 
        for p in place : 
            if p.startswith('deliver') or p.startswith('Deliver') or p.startswith('Radio') or p.startswith('Broadcast') or p.startswith('broadcast') or p.startswith('presented') or p.startswith('Air') or p.startswith('original') or p.startswith('Paper') : 
                deliveredAtCleaned.append(p)  



print(len(deliveredAtCleaned))

100


In [23]:
# Creating DF
df = pd.DataFrame(list(zip(nameList, findTitles(soup), mainlinks, deliveredAtCleaned )),
               columns =['Speaker', 'Title', 'Links', "Delivered"])

df.head(11)

,Speaker,Title,Links,Delivered
0,"Martin Luther King, Jr.",I Have A Dream,speeches/mlkihaveadream.htm,"delivered 28 August 1963, at the Lincoln Memor..."
1,John Fitzgerald Kennedy,Inaugural Address,speeches/jfkinaugural.htm,"delivered 20 January 1961, Washington, D.C."
2,Franklin Delano Roosevelt,First Inaugural Address,speeches/fdrfirstinaugural.html,Delivered 4 March 1933
3,Franklin Delano Roosevelt,Pearl Harbor Address to the Nation,speeches/fdrpearlharbor.htm,"delivered 8 December 1941, Washington, D.C."
4,Barbara Charline Jordan,1976 DNC Keynote Address,speeches/barbarajordan1976dnc.html,"delivered 12 July 1976, New York, NY"
5,Richard Milhous Nixon,Checkers,speeches/richardnixoncheckers.html,delivered and broadcast live on television 23 ...
6,Malcolm X,The Ballot or the B,http://americanradioworks.publicradio.org/feat...,None
7,Ronald Wilson Reagan,Shuttle 'Challenger' Disaster Address,speeches/ronaldreaganchallenger.htm,delivered 28 January 1986
8,John Fitzgerald Kennedy,Houston Ministerial Association,speeches/jfkhoustonministers.html,delivered 12 September 1960 at the Rice Hotel ...
9,Lyndon Baines Johnson,We Shall Overcome,speeches/lbjweshallovercome.htm,"delivered 15 March 1965, Washington, D.C."


Test script for first speech, will replace URL with iteration over URL list mainlinks.

In [16]:
URL='https://www.americanrhetoric.com/'+mainlinks[0]
Page=requests.get(URL, headers={'user-agent': 'Mozilla/5.0'})
speechsoup=BeautifulSoup(Page.content, 'html.parser')
script=[p.get_text() for p in speechsoup.find_all("font",{'face':'Verdana','size':2})]
script=' '.join([str(line) for line in script])
script=' '.join(script.split())



TypeError: list indices must be integers or slices, not tuple

Script iterating over the different URLs of the AmericanRhetoric site. First results arent looking too great... It seems that the site uses diferent HTML formatting for many of the URLs, meaning I'll have to adjust the way I select the text from the sites.

In [4]:
speeches=[None]*95
for d in [6,57,77,79,84]:
    del mainlinks[d]
for l in range(0,94):
    URL='https://www.americanrhetoric.com/'+mainlinks[l]
    Page=requests.get(URL, headers={'user-agent': 'Mozilla/5.0'})
    speechsoup=BeautifulSoup(Page.content, 'html.parser')
    script=[p.get_text() for p in speechsoup.find_all("font",{'face':'Verdana','size':2})]
    script=' '.join([str(line) for line in script])
    script=' '.join(script.split())
    speeches[l]=script


In [179]:
#speeches

['I am happy to join with you today in what will go down in history as the greatest demonstration for freedom in the history of our nation. Five score years ago, a great American, in whose symbolic shadow we stand today, signed the Emancipation Proclamation. This momentous decree came as a great beacon light of hope to millions of Negro slaves who had been seared in the flames of withering injustice. It came as a joyous daybreak to end the long night of their captivity. But one hundred years later, the Negro still is not free. One hundred years later, the life of the Negro is still sadly crippled by the manacles of segregation and the chains of discrimination. One hundred years later, the Negro lives on a lonely island of poverty in the midst of a vast ocean of material prosperity. One hundred years later, the Negro is still languished in the corners of American society and finds himself an exile in his own land. And so we\'ve come here today to dramatize a shameful condition. In a sen